This jupyter notebook was made to show how one would redo what was accomplished in `https://github.com/hjlvdheiden/RobotLab/tree/main/main` using `preprocessor` and `BayesianOptimization`.

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
from preprocessing import preprocessor
from bayesian_opt import BayesianOptimization

In [3]:
raw_df = pd.read_excel('internship_simplified.xlsx',sheet_name='Sheet1')
settings_df = pd.read_excel('internship_simplified.xlsx',sheet_name='Sheet2')
raw_df.head(3)

,Time,T° Read 1:472,A1,A2,A3,A4,A5,A6,A7,A8,...,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12
0,00:00:07,25.1,0.0696,0.0789,0.0764,0.0753,0.0709,0.0732,0.0753,0.0723,...,0.0694,0.0749,0.0786,0.0793,0.0719,0.0757,0.0664,0.0764,0.0718,0.0724
1,00:01:07,25.0,0.0711,0.0757,0.0745,0.0743,0.0739,0.0755,0.0717,0.0731,...,0.0702,0.0783,0.0821,0.0825,0.0725,0.0758,0.0684,0.0780,0.0727,0.0724
2,00:02:07,25.0,0.0734,0.0781,0.0773,0.0762,0.0753,0.0789,0.0712,0.0725,...,0.0704,0.0804,0.0860,0.0862,0.0722,0.0760,0.0722,0.0783,0.0733,0.0727


In [4]:
settings_df.head(3)

,well,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12
0,pH,6.398,6.096,5.836,6.104,6.024,6.478,6.140,5.598,6.036,...,4.568,4.930,5.872,5.012,6.488,5.728,5.450,5.216,5.392,4.740
1,c1,0.820,0.051,0.412,0.388,0.282,0.528,0.977,0.486,0.543,...,0.310,0.329,0.281,0.635,0.577,0.820,0.399,0.816,0.343,0.015
2,c2,1.512,1.180,1.234,1.250,0.108,0.580,0.570,1.222,0.218,...,1.314,0.206,0.490,0.808,0.858,1.746,1.752,0.338,0.960,0.164


In [5]:
# We do not have implemented functionality for special_wells otherwise known as override_wells.
processed_df = preprocessor(settings_df, raw_df, pos_wells=["G5","G9"], plot=False, return_coef=False)
processed_df.head(3)

c:\Users\danin\Downloads\ict-f0ntys-internship\example_use\..\data_generation.py:9: RuntimeWarning: overflow encountered in exp
  return a * np.exp(b * t)


,pH,c1,c2,norm_yield_grad,max_yield,function_type
0,6.398,0.820,1.512,0.268456,0.2296,linear
1,6.096,0.051,1.180,0.306807,0.2376,linear
2,5.836,0.412,1.234,0.297220,0.2492,linear


Here `function_type` column could be used as categorical input for suggesting new candidates, but we won't use it in this notebook.

In [13]:
processed_df = processed_df[['pH','c1','c2','norm_yield_grad']]
processed_df

,pH,c1,c2,norm_yield_grad
0,6.398,0.820,1.512,0.268456
1,6.096,0.051,1.180,0.306807
2,5.836,0.412,1.234,0.297220
3,6.104,0.388,1.250,0.249281
4,6.024,0.282,0.108,0.297220
...,...,...,...,...
91,5.728,0.820,1.746,0.402685
92,5.450,0.399,1.752,0.364334
93,5.216,0.816,0.338,0.153404
94,5.392,0.343,0.960,0.086290


Applying 1 iteration of bayesian optimization.

In [ ]:
bo_model = BayesianOptimization().fit(X=processed_df,y=['norm_yield_grad'])
# Here the bounds don't match those from the Github attempt. 
# These were only chosen to show that this functionality also exists in our code.
candidates = bo_model.candidates(q=10, acq_func_name='LogEI', bounds = [[1e-5, 1e-5, 1e-5],[14, 1, 2]],
                                 export_df=True, q_sampling_method='Believer')

In [15]:
candidates

,pH,c1,c2,norm_yield_grad
0,6.67,0.00,0.68,1.515166
1,4.40,0.03,0.88,1.142575
2,4.24,0.79,0.80,0.934114
3,4.12,0.34,0.73,0.892416
4,6.54,0.83,1.94,1.768247
5,4.49,0.12,1.87,2.893888
6,11.70,0.39,1.78,0.402696
7,0.00,1.00,0.00,0.402696
8,6.80,0.13,0.53,1.296123
9,14.00,0.00,0.00,0.411045
